In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/falaaharifkhan/Documents/fairness-variance


## Import dependencies

In [4]:
import os
import pandas as pd
from IPython.display import display
from tqdm import tqdm

from configs import config
from configs.constants import ModelSetting
from utils.analyzers.subgroups_variance_analyzer import SubgroupsVarianceAnalyzer
from utils.common_helpers import create_tuned_base_model, save_metrics_to_file
from utils.custom_classes.data_loader import CompasWithoutSensitiveAttrsDataset
from utils.custom_classes.generic_pipeline import GenericPipeline
from utils.analyzers.subgroups_statistical_bias_analyzer import SubgroupsStatisticalBiasAnalyzer

## Configs

In [5]:
DATASET_NAME = "COMPAS_Without_Sensitive_Attributes"
EXPERIMENT_NAME = 'Hypothesis_Space'

seed_lst = [101, 111, 201, 221, 311, 333, 410, 555, 666, 851, 999, 1010]
SEX_priv = 0
RACE_priv = 'Caucasian'
N_ESTIMATORS = 200
SENSITIVE_ATTRIBUTES = ['sex', 'race']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('results', 'models_tuning', f'tuning_results_{DATASET_NAME}_20230117__012024.csv')

## Models tuned hyper-parameters

In [6]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

,Dataset_Name,Model_Name,F1_Score,Accuracy_Score,Model_Best_Params
0,COMPAS_Without_Sensitive_Attributes,LogisticRegression,0.6785,0.6837,"{'C': 1, 'max_iter': 50, 'penalty': 'l2', 'sol..."
1,COMPAS_Without_Sensitive_Attributes,DecisionTreeClassifier,0.6952,0.6989,"{'criterion': 'gini', 'max_depth': 20, 'max_fe..."
2,COMPAS_Without_Sensitive_Attributes,RandomForestClassifier,0.6962,0.7008,"{'max_depth': 4, 'max_features': 0.6, 'min_sam..."
3,COMPAS_Without_Sensitive_Attributes,XGBClassifier,0.6971,0.7008,"{'lambda': 100, 'learning_rate': 0.1, 'max_dep..."
4,COMPAS_Without_Sensitive_Attributes,KNeighborsClassifier,0.7056,0.7112,"{'metric': 'minkowski', 'n_neighbors': 25, 'we..."
5,COMPAS_Without_Sensitive_Attributes,MLPClassifier,0.6834,0.6875,"{'activation': 'relu', 'hidden_layer_sizes': (..."


## Load dataset

In [7]:
dataset = CompasWithoutSensitiveAttrsDataset(dataset_path='data/COMPAS.csv')
dataset.X_data.head()

,juv_fel_count,juv_misd_count,juv_other_count,priors_count,age_cat_25 - 45,age_cat_Greater than 45,age_cat_Less than 25,c_charge_degree_F,c_charge_degree_M
0,0.0,-2.340451,1.0,-15.010999,1,0,0,0,1
1,0.0,0.000000,0.0,0.000000,1,0,0,1,0
2,0.0,0.000000,0.0,0.000000,0,0,1,1,0
3,0.0,0.000000,0.0,6.000000,1,0,0,0,1
4,0.0,0.000000,0.0,7.513697,1,0,0,1,0


## Run experiments

In [8]:
def create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed):
    base_pipeline = GenericPipeline(dataset, sensitive_attributes, priv_values)
    _ = base_pipeline.create_train_test_split_without_sensitive_attrs(dataset, config.TEST_SET_FRACTION, seed=model_seed)

    print('\nProtected groups splits:')
    for g in base_pipeline.test_groups.keys():
        print(g, base_pipeline.test_groups[g].shape)

    return base_pipeline


def get_model_metrics(base_model, n_estimators, dataset, sensitive_attributes, priv_values, model_seed_lst,
                      dataset_name, base_model_name, exp_num=1):
    
    results_over_all_runs = pd.DataFrame({})
    
    for model_seed in model_seed_lst:
        base_pipeline = create_base_pipeline(dataset, sensitive_attributes, priv_values, model_seed)
        print('\n\nX train and validation set: ')
        display(base_pipeline.X_train_val.head(10))

        # Compute variance metrics for subgroups
        save_results = False
        stability_fairness_analyzer = SubgroupsVarianceAnalyzer(ModelSetting.BATCH, n_estimators, base_model, base_model_name,
                                                                base_pipeline.X_train_val, base_pipeline.y_train_val,
                                                                base_pipeline.X_test, base_pipeline.y_test,
                                                                base_pipeline.sensitive_attributes, base_pipeline.priv_values, base_pipeline.test_groups,
                                                                base_pipeline.target, dataset_name)

        y_preds, variance_metrics_df = stability_fairness_analyzer.compute_metrics(save_results=save_results,
                                                                                   result_filename=None,
                                                                                   save_dir_path=None,
                                                                                   make_plots=False)

        # Compute bias metrics for subgroups
        bias_analyzer = SubgroupsStatisticalBiasAnalyzer(base_pipeline.X_test, base_pipeline.y_test,
                                                         base_pipeline.sensitive_attributes, base_pipeline.priv_values,
                                                         base_pipeline.test_groups)
        dtc_res = bias_analyzer.compute_subgroups_metrics(y_preds,
                                                          save_results=False,
                                                          result_filename=None,
                                                          save_dir_path=None)
        bias_metrics_df = pd.DataFrame(dtc_res)
        metrics_df = pd.concat([variance_metrics_df, bias_metrics_df])
        metrics_df['SEED'] = [model_seed]*len(metrics_df)
        
        if len(results_over_all_runs) == 0:
            print("saving first")
            results_over_all_runs = metrics_df.deep_copy=True
        else:
            results_over_all_runs = pd.concat([results_over_all_runs, metrics_df], ignore_index=True, copy=True)

    # Save metrics
    result_filename = f'{EXPERIMENT_NAME}_Metrics_{dataset_name}_Experiment_{exp_num}_{base_model_name}'
    save_dir_path = os.path.join('results', 'hypothesis_space')
    save_metrics_to_file(results_over_all_runs, result_filename, save_dir_path)

    return results_over_all_runs

In [9]:
def run_experiment(exp_num: int, model_seed_lst: int):
    """
    Find variance and bias metrics for each model in config.MODELS_CONFIG.
    Save results in results/config.MODELS_CONFIG folder.

    :param exp_num: the number of experiment; is used to name the result file with metrics
    """
    all_results = {}
    for model_idx in tqdm(range(len(config.MODELS_CONFIG))):
        print('#' * 30, f' [Experiment {exp_num}] Analyze {config.MODELS_CONFIG[model_idx]["model_name"]} ', '#' * 30)
        #model_seed += 1
        try:
            base_model = create_tuned_base_model(config.MODELS_CONFIG[model_idx]['model'],
                                                 config.MODELS_CONFIG[model_idx]['model_name'],
                                                 models_tuned_params_df)
            results_df = get_model_metrics(base_model, N_ESTIMATORS, dataset, SENSITIVE_ATTRIBUTES, PRIV_VALUES,
                                           model_seed_lst=model_seed_lst,
                                           dataset_name=DATASET_NAME,
                                           base_model_name=config.MODELS_CONFIG[model_idx]['model_name'],
                                           exp_num=exp_num)
            print(f'\n[Experiment {exp_num}] Metrics confusion matrix:')
            #display(results_df)
            all_results[config.MODELS_CONFIG[model_idx]['model_name']] = results_df
        except Exception as err:
            print(f'ERROR with {config.MODELS_CONFIG[model_idx]["model_name"]}: ', err)

        print('\n\n\n')
    return all_results


### Experiment 1

In [10]:
# TOD: add dataset as a parameter
all_results_df = run_experiment(exp_num=1, model_seed_lst=seed_lst)

  0%|                                                     | 0/6 [00:00<?, ?it/s]

##############################  [Experiment 1] Analyze LogisticRegression  ##############################
Baseline X_train shape:  (4222, 9)
Baseline X_test shape:  (1056, 9)

Protected groups splits:
sex_race_priv (88, 11)
sex_race_dis (518, 11)
sex_priv (199, 11)
sex_dis (857, 11)
race_priv (427, 11)
race_dis (629, 11)


X train and validation set: 


,juv_other_count,age_cat_25 - 45_1,priors_count,c_charge_degree_F_1,age_cat_Greater than 45_0,age_cat_Greater than 45_1,juv_fel_count,c_charge_degree_M_1,age_cat_Less than 25_0,c_charge_degree_F_0,juv_misd_count,c_charge_degree_M_0,age_cat_Less than 25_1,age_cat_25 - 45_0
470,-0.148179,1,0.106154,1,1,0,-0.098697,0,1,0,-0.127240,1,0,0
1328,-0.148179,1,2.409132,1,1,0,-0.098697,0,1,0,0.348324,1,0,0
3551,-0.148179,1,0.620185,0,1,0,-0.098697,1,1,1,-0.127240,0,0,0
1628,-0.148179,1,-0.085761,0,1,0,-0.098697,1,1,1,-0.127240,0,0,0
4138,1.111816,1,-0.280851,1,1,0,-0.098697,0,1,0,-0.127240,1,0,0
5030,-0.148179,1,0.693041,1,1,0,-0.098697,0,1,0,-0.127240,1,0,0
4510,-0.148179,1,-0.085761,0,1,0,-0.098697,1,1,1,-0.127240,0,0,0
1676,-0.148179,0,-0.661506,1,1,0,-0.098697,0,0,0,-0.127240,1,1,1
5047,-0.148179,1,-0.469591,0,1,0,-0.098697,1,1,1,-0.127240,0,0,0
1554,-0.148179,0,-0.668317,0,0,1,-0.098697,1,1,1,-0.127240,0,0,1


2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 1 / 200
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Classifier 1 / 200 was tested
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 2 / 200
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Classifier 2 / 200 was tested
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 3 / 200
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Classifier 3 / 200 was tested
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 4 / 200
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Classifier 4 / 200 was tested
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 5 / 200
2023-01-17 01:55:53 abstract_overall_variance_analyzer.py INFO    : Classifier 5 / 200 w

2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Classifier 41 / 200 was tested
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 42 / 200
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Classifier 42 / 200 was tested
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 43 / 200
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Classifier 43 / 200 was tested
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 44 / 200
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Classifier 44 / 200 was tested
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 45 / 200
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Classifier 45 / 200 was tested
2023-01-17 01:55:54 abstract_overall_variance_analyzer.py INFO    : Start testing of 

...

2023-01-17 02:54:56 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 122 / 200
2023-01-17 02:55:02 abstract_overall_variance_analyzer.py INFO    : Classifier 122 / 200 was tested
2023-01-17 02:55:02 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 123 / 200
2023-01-17 02:55:08 abstract_overall_variance_analyzer.py INFO    : Classifier 123 / 200 was tested
2023-01-17 02:55:08 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 124 / 200
2023-01-17 02:55:14 abstract_overall_variance_analyzer.py INFO    : Classifier 124 / 200 was tested
2023-01-17 02:55:14 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 125 / 200
2023-01-17 02:55:18 abstract_overall_variance_analyzer.py INFO    : Classifier 125 / 200 was tested
2023-01-17 02:55:18 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 126 / 200
2023-01-17 02:55:23 abstract_overall_variance_analyzer.py INFO    : Cl

2023-01-17 02:58:30 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 162 / 200
2023-01-17 02:58:35 abstract_overall_variance_analyzer.py INFO    : Classifier 162 / 200 was tested
2023-01-17 02:58:35 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 163 / 200
2023-01-17 02:58:41 abstract_overall_variance_analyzer.py INFO    : Classifier 163 / 200 was tested
2023-01-17 02:58:41 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 164 / 200
2023-01-17 02:58:46 abstract_overall_variance_analyzer.py INFO    : Classifier 164 / 200 was tested
2023-01-17 02:58:46 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 165 / 200
2023-01-17 02:58:51 abstract_overall_variance_analyzer.py INFO    : Classifier 165 / 200 was tested
2023-01-17 02:58:51 abstract_overall_variance_analyzer.py INFO    : Start testing of classifier 166 / 200
2023-01-17 02:58:57 abstract_overall_variance_analyzer.py INFO    : Cl



##############################  Stability metrics  ##############################
General Ensemble Accuracy: 0.6146
Mean: 0.5285
Std: 0.0243
IQR: 0.0317
Entropy: 0.0
Jitter: 0.0625
Per sample accuracy: 0.6215
Label stability: 0.9101




100%|██████████████████████████████████████████| 6/6 [1:07:37<00:00, 676.32s/it]

ERROR with MLPClassifier:  object of type 'bool' has no len()






In [12]:
all_results_df

{}